# Viewing TLE data

This [Jupyter](https://jupyter.org) notebook explores and analyzes the raw [Two-line Element Set (TLE)](https://en.wikipedia.org/wiki/Two-line_element_set) satellite tracking data provided for the 12/2020 VAULT technical scenario, showing what data is available and how it can be accessed and visualized from Python. The notebook also acts as a runnable application that can be put on a server to allow users to explore the raw data interactively.

In [ ]:
import time, datetime, calendar
import pandas as pd
import numpy as np
import colorcet as cc
import panel as pn
import datetime as dt
import holoviews as hv
from holoviews.operation.datashader import rasterize
import skyfield
from skyfield.api import EarthSatellite, load
from skyfield.framelib import itrs
hv.extension('bokeh')

First, let's load some 2017 TLE data using [pandas](https://pandas.pydata.org):

In [ ]:
tle = pd.read_csv('data/Cleaned_TLE/tle2017.csv')
tle.head(2)

We'll need to work with the `tle` field, where each entry is a three-line string, with the 2nd and 3rd lines containing the data about the satellite position, heading, etc.

In [ ]:
tle.iloc[0]['tle']

Let's convert that to a list of string pairs (2nd and 3rd lines):

In [ ]:
%%time
new_lines = [el.replace('None\n', '').split('\n')[:2] for el in tle['tle']] # Splitting the file

In [ ]:
new_lines[:5]

Given a TLE string pair, we can use the [skyfield](https://rhodesmill.org/skyfield) library to calculate the longitude and latitude location of the satellite (projected down on the earth's surface) at the time of the TLE tracking record entry. Note that the calculation takes some time, for a large file.

In [ ]:
def modulo_lon(val):
    return (val+180) % 360 - 180
    
def compute_lat_lon(line1, line2):
    """Get the Lat/Lon at the TLE epoch"""
    sat = EarthSatellite(line1, line2)
    lat, lon, _ = sat.at(sat.epoch).frame_latlon(itrs)
    return lat.degrees, lon.degrees

def lat_lon_from_lines(lines):
    lons, lats = [], []
    for line1, line2 in lines:
        lat, lon = compute_lat_lon(line1,line2)
        if None not in [lon, lat]:
            lons.append(modulo_lon(lon))
            lats.append(lat)
    return np.array(lats), np.array(lons)

In [ ]:
%%time
lines=20000
lines=None # comment out for quick run
lats, lons = lat_lon_from_lines(new_lines[:lines])

Once we have the lat,lon data, we can convert it to easting,northing for plotting with [Datashader](https://datashader.org) and [HoloViews](https://holoviews.org) on a web tile map:

In [ ]:
%%time
eastings, northings = hv.util.transform.lon_lat_to_easting_northing(lons, lats)

In [ ]:
tiles  = hv.element.tiles.ESRI().opts(alpha=0.5, bgcolor='black') 
points = rasterize(hv.Points((eastings,northings)))
points = points.opts(width=900, height=600, cmap=cc.kbc[64:], cnorm='eq_hist')
tiles * points

We can see lots of interesting structure in the locations of the satellites at the epoch time of each record. To help make sense of it, let's overlay data about the [locations of dedicated and collateral tracking radar stations](https://en.wikipedia.org/wiki/Solid_State_Phased_Array_Radar_System) in the [Space Surveillance Network](https://en.wikipedia.org/wiki/File:Space_Surveillance_Network.jpg):

In [ ]:
stations = {"Beale":       (39.136111, -121.436389),
            "Cape Cod":    (41.753333,  -70.538611),
            "Clear":       (64.290556, -149.187222),
            "Thule":       (76.567850,  -68.284214),
            "Fylingdales": (54.358889,   -0.669722),
            "Eglin":       (30.489444,  -86.542222),
            "Cavalier":    (48.731944,  -97.904444),
            "Ascension":   (-7.969444,  -14.393889),
            "Cobra Dane":  (52.712222,  174.113611)}

stations = {k:hv.util.transform.lon_lat_to_easting_northing(v[1],v[0]) for k,v in stations.items()}

In [ ]:
radar = hv.Points(list(stations.values())).opts(color='white', size=5)
names = hv.Labels({('x', 'y'): list(stations.values()), 
                    'text': list(stations.keys())}, ['x', 'y'], 'text')
names = names.opts(text_font_size='10pt', yoffset=0.5e6, text_color="white")

In [ ]:
tiles  = hv.element.tiles.ESRI().opts(alpha=0.5, bgcolor='black') 
points = rasterize(hv.Points((eastings,northings))).opts(width=900, height=600, cmap=cc.kbc[64:200], cnorm='eq_hist')
tiles * points * radar * names

Now we can see that most of the records (at the epoch) are from when the satellites were close to one of the radar-based monitoring stations indicated, apart from those along the equator (presumably in geostationary orbit) and a few other widely distributed observations.

Finally, we'll package up the above plot into a servable app using [Panel](https://panel.holoviz.org):

In [ ]:
pn.Column("# Raw TLE data", 
          "TLE records plotted at the epoch time in geo coordinates, "
          "with Space Surveillance Network radar stations overlaid",
          tiles * points * radar * names).servable();